In [ ]:
!pip3 install transformers

In [ ]:
import os
import time
from enum import Enum

from tqdm import tqdm
import yaml

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

In [ ]:
os.getcwd()

'/content'

In [ ]:
torch.cuda.is_available()

True

In [ ]:
def load_config(config_file):
    with open(os.path.abspath(config_file)) as f:
        config = yaml.safe_load(f)
    return config


def one_hot(a):
    b = np.zeros((a.shape[0], a.max() + 1))
    b[np.arange(a.shape[0]), a] = 1
    return b

In [ ]:
class TokenizerType(Enum):
    bert_tokenizer = 'bert_tokenizer'


TOKENIZERS_CLASSES = {
    TokenizerType.bert_tokenizer: BertTokenizer
}


class ModelType(Enum):
    bert_for_sequence_classification = 'bert_for_sequence_classification'


MODELS_CLASSES = {
    ModelType.bert_for_sequence_classification: BertForSequenceClassification
}

In [ ]:
class Evaluator:
    def __init__(self, config_file):
        self.input_label = 'input'
        self.target_label = 'target'
        self.prediction_label = 'predictions'

        config = load_config(config_file)
        self.device = config['model']['device']
        self.device = torch.device('cuda' if torch.cuda.is_available() and self.device == 'gpu' else 'cpu')
        print('Device = {}'.format(self.device))
        if self.device == torch.device('cuda'):
            torch.cuda.empty_cache()

        self._setup_output_config(config)
        self._setup_data_config(config)
        self._setup_model_config(config)

    def _setup_data_config(self, config):
        pass

    def _setup_model_config(self, config):
        pass

    def _setup_output_config(self, config):
        pass

    def _save_prediction(self, input, targets, predictions, output_path):
        prediction_df = pd.DataFrame(
            data=
            {
                self.input_label: input,
                self.target_label: targets,
                self.prediction_label: predictions
            }
        )
        prediction_df.to_csv(output_path, sep=',', index=False)

In [ ]:
class Predictor(Evaluator):
    def __init__(self, config_file):
        super(Predictor, self).__init__(config_file)

    def _setup_data_config(self, config):
        self.test_df = pd.read_csv(config['dataset']['test_path'], sep=',')
        self.input_label = config['dataset'].get('input_label', self.test_df.columns[0])
        self.target_label = config['dataset'].get('target_label', self.test_df.columns[-1])
        self.prediction_label = config['dataset'].get('target_label', 'prediction')

        self.tokenizer = TOKENIZERS_CLASSES[TokenizerType[config['tokenizer']['type']]].from_pretrained(
            config['tokenizer']['name'], do_lower_case=config['tokenizer']['do_lower_case']
        )
        encoded_test_data = self.tokenizer.batch_encode_plus(
            self.test_df[self.input_label].values,
            add_special_tokens=config['tokenizer']['add_special_tokens'],
            return_attention_mask=config['tokenizer']['return_attention_mask'],
            pad_to_max_length=config['tokenizer']['pad_to_max_length'],
            max_length=config['tokenizer']['seq_length'],
            return_tensors=config['tokenizer']['return_tensors']
        )
        self.test_input_ids = encoded_test_data['input_ids']
        self.test_attention_masks = encoded_test_data['attention_mask']

    def _setup_model_config(self, config):
        self.model = MODELS_CLASSES[ModelType[config['model']['type']]].from_pretrained(
            config['model']['name'],
            num_labels=config['model'].get(
                'num_label',
                len(np.unique(self.test_df[self.target_label].values))
            ),
            output_attentions=False,
            output_hidden_states=False
        )
        self.model.to(self.device)
        self.model.load_state_dict(
            torch.load(
                config['model']['path'], map_location=torch.device(self.device)
            )
        )

    def _setup_output_config(self, config):
        self.prediction_path = config['res']['prediction_path']
        os.makedirs(self.prediction_path, exist_ok=True)
        self.metrics_path = config['res']['metrics_path']
        os.makedirs(self.metrics_path, exist_ok=True)
        self.all_test_outputs = list()

    def predict(self):
        predict_start = time.perf_counter()
        for input_ids, attention_mask in tqdm(
                zip(self.test_input_ids, self.test_attention_masks), desc='Prediction'
        ):
            input_ids, attention_mask = input_ids.unsqueeze(0).to(self.device), attention_mask.unsqueeze(0).to(self.device)
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            self.all_test_outputs.append(preds.detach().cpu().numpy())
        predict_end = time.perf_counter()
        self.all_test_outputs = np.concatenate(self.all_test_outputs, axis=0)

        accuracy = accuracy_score(self.test_df[self.target_label].values, self.all_test_outputs)

        conf_matrix = confusion_matrix(
            self.test_df[self.target_label].values,
            self.all_test_outputs,
            labels=np.sort(np.unique(self.test_df[self.target_label].values)))
        conf_matrix_display = ConfusionMatrixDisplay(
            conf_matrix,
            display_labels=np.sort(np.unique(self.test_df[self.target_label].values))
        )
        conf_matrix_display.plot()
        conf_matrix_display.figure_.savefig(
            os.path.join(self.metrics_path, 'confusion_matrix.png'), dpi=conf_matrix_display.figure_.dpi
        )

        report = classification_report(
            self.test_df[self.target_label].values,
            self.all_test_outputs, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        report_df.to_csv(os.path.join(self.metrics_path, 'classification_report.csv'), sep=',', index=False)

        print('Accuracy: {0:.7f}, time: {1:.7f}, mean time: {2:.7f}'.format(
            accuracy,
            predict_end - predict_start,
            predict_end - predict_start / len(self.test_input_ids))
        )

        self._save_prediction(
            self.test_df[self.input_label].values,
            self.test_df[self.target_label].values,
            self.all_test_outputs,
            os.path.join(self.prediction_path, 'test.csv')
        )

In [ ]:
config_file = '/content/config(colab).yml'
trainer = Predictor(config_file)
trainer.predict()